### to run this code, you need to have standalone EMBOSS (getorf), BLAST, and HMMER installed

In [1]:
import subprocess
import os
from glob import glob
from Bio import SearchIO
from Bio import SeqIO

## load and adjust all paths

In [2]:
outdir = "/home/majnusova/all/projects/plv/data/eustig_orfs/" #adjust
genomes = glob("/home/majnusova/all/projects/plv/data/genomes_test/*.fasta")
hmm_model = "/home/majnusova/all/projects/bilabrum/data/hmm/Packiging_ATPase.hmm" #adjust
output_directory = "/home/majnusova/all/projects/plv/data/eustigs_hmmout/" #adjust
orf_files = glob("/home/majnusova/all/projects/plv/data/eustig_orfs/*_orfs.fa") #adjust: list of orf databases
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"

### Extract ORFs from genomes

In [72]:
def run_getorf(genome, orf_file):
    command = ["getorf", "-sequence", genome, "-outseq", orf_file, "-table", "0", "-minsize", "360", "-find", "1"] #adjust (120 aa, START to STOP, standard gen. code)
    subprocess.run(command, check=True)

outdir = "/home/majnusova/all/projects/plv/data/eustig_orfs/" #adjust
genomes = glob("/home/majnusova/all/projects/plv/data/genomes_test/*.fasta") #adjust - this is just a test dataset

for genome in genomes:
    orf_file = f"{outdir}{genome.split('/')[-1].replace('.fasta', '_orfs.fa')}"
    run_getorf(genome, orf_file)

Find and extract open reading frames (ORFs)


KeyboardInterrupt: 

### hmmsearch with HMM against those extracted ORFs -> hmmout

In [ ]:
# running hhmmsearch against multiple databases at once and saving the resulting files using the .hmmout suffix
def run_hmmsearch(orf_file, hmm_model, output_file):
    command = ["hmmsearch", "-o", output_file, hmm_model, orf_file]
    subprocess.run(command, check=True)

hmm_model = "/home/majnusova/all/projects/bilabrum/data/hmm/Packiging_ATPase.hmm" #adjust
output_directory = "/home/majnusova/all/projects/plv/data/eustigs_hmmout/" #adjust
orf_files = glob("/home/majnusova/all/projects/plv/data/eustig_orfs/*_orfs.fa") #adjust: list of orf databases


# iterate over the list of database files and run hmmsearch 
for orf_file in orf_files:
    output_file = f"{output_directory}{orf_file.split('/')[-1].replace('_orfs.fa', '_atp.hmmout')}" 
    run_hmmsearch(orf_file, hmm_model, output_file) #needs to be nested inside the loop!


### saving IDs of ALL homologs above inclusion threshold found by HMMER and name of the assembly they were found in
### genome_scaffold_ids - genome: ids

In [ ]:
# creating a dict of lists (genomes: ids)
genome_scaffold_ids = {}

for file in glob("/home/majnusova/all/projects/plv/data/eustigs_hmmout/*_atp.hmmout"):
    genome_name = os.path.basename(file).split('_atp.hmmout')[0] # getting the real names of the genomes; os.path.basename(file) extracts the base name from the file's full path.
    if genome_name not in genome_scaffold_ids:
        genome_scaffold_ids[genome_name] = []

    hmmer_file = SearchIO.read(file, "hmmer3-text")
    
    for record in hmmer_file:
        if record.is_included:
            scaffold_id = record.id.rsplit("_", 1)[0] # scaffold ID - removing _orf
            genome_scaffold_ids[genome_name].append(scaffold_id)

In [24]:
genome_scaffold_ids

{'Chic10_scaffolds': ['NODE_5080_length_5696_cov_26.6876',
  'NODE_381_length_27838_cov_14.2215',
  'NODE_7293_length_4272_cov_22.9291',
  'NODE_5397_length_5445_cov_37.4879',
  'NODE_6066_length_4954_cov_14.3046',
  'NODE_847_length_17049_cov_12.9829',
  'NODE_7343_length_4249_cov_22.2454',
  'NODE_19673_length_1550_cov_34.6107',
  'NODE_1409_length_12728_cov_15.5927',
  'NODE_1574_length_11923_cov_14.5027',
  'NODE_711_length_18631_cov_14.7806',
  'NODE_15445_length_2031_cov_23.9084',
  'NODE_15112_length_2083_cov_16.6144',
  'NODE_12470_length_2567_cov_17.4088',
  'NODE_34064_length_837_cov_18.9655',
  'NODE_4057_length_6629_cov_27.157',
  'NODE_865_length_16875_cov_12.6059',
  'NODE_869_length_16855_cov_12.4798',
  'NODE_6477_length_4696_cov_21.0899',
  'NODE_17368_length_1781_cov_15.3621',
  'NODE_18418_length_1671_cov_21.013',
  'NODE_352_length_29307_cov_12.7034',
  'NODE_31105_length_924_cov_16.8895',
  'NODE_4114_length_6570_cov_18.8181',
  'NODE_64132_length_412_cov_37.521',


In [25]:
total_ids = sum(len(id) for id in genome_scaffold_ids.values())
total_ids

280

### Extracting scaffolds with the viral homologs found by HMMER (ids_list), scaffolds shorter than 18000 nt are discarded

#### ids_list_scaffolds = all viral scaffolds above inclusion threshold (may be shorter than 20000)

#### to be able to extract the scaffolds, blastable databases need to be created first (for genome and orfs)

In [113]:
def makeblastdb(file_path, db_type, blastables_outdir):
    db_name = f"{blastables_outdir}/{file_path.split('/')[-1].split('.')[0]}"
    command = ["makeblastdb", "-in", file_path, "-dbtype", db_type, "-out", db_name, "-parse_seqids"]
    subprocess.run(command, check=True)

blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"


for genome_path in genomes:
    makeblastdb(genome_path, "nucl", blastables_outdir) # misto file_path volame s genome_path

for orf_file_path in orf_files:
    makeblastdb(orf_file_path, "prot", blastables_outdir)




Building a new DB, current time: 01/11/2024 17:32:41
New DB name:   /home/majnusova/all/projects/plv/data/blastables/Chic10_scaffolds
New DB title:  /home/majnusova/all/projects/plv/data/genomes_test/Chic10_scaffolds.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 758531 sequences in 9.94234 seconds.




Building a new DB, current time: 01/11/2024 17:32:53
New DB name:   /home/majnusova/all/projects/plv/data/blastables/Vischeria_C74_genome_v1
New DB title:  /home/majnusova/all/projects/plv/data/genomes_test/Vischeria_C74_genome_v1.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 55 sequences in 0.476354 seconds.




Building a new DB, current time: 01/11/2024 17:32:54
New DB name:   /home/majnusova/all/projects/plv/data/blastables/Characiopsis_New_scaffolds
New DB title:  /home/majnusova/all/projects/plv/data/genomes_test/Characiopsis_New_scaffolds.fasta
Sequ

### extracting all viral scaffolds present in ids_list_scaffolds and deleting those shorter than 20000
# this code needs to be rewritten, it works but its repetitive 

In [17]:
scaffolds_of_interest = []
def run_blastdbcmd(id, genome_db, outscaffold):
    command = ["blastdbcmd", "-entry", str(id), "-db", genome_db, "-out", outscaffold]
    result = subprocess.run(command, text=True, capture_output=True)

    # Check if blastdbcmd executed successfully
    if result.returncode != 0: # this means that error occured during the execution of the command
        print(f"Warning: blastdbcmd failed for ID {id} in database {genome_db}.")
        os.remove(outscaffold)
        return False

    # Proceed to check if the scaffold file exists and its size
    if os.path.exists(outscaffold) and os.path.getsize(outscaffold) > 0:
        record = SeqIO.read(outscaffold, "fasta")
        if len(record.seq) < 20000:
            print(f"Notice: Scaffold {id} is shorter than 20000 nucleotides, removing file.")
            os.remove(outscaffold)
            return False
        else:
            scaffolds_of_interest.append(id)  
            return True
    else:
        print(f"Notice: No scaffold file created for ID {id}.")
        return False
scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
prot_database = "_orfs."

for genome, ids in genome_scaffold_ids.items():
    for id in ids:
        for filename in os.listdir(blastables_outdir):
            if prot_database not in filename:
                base_name = filename.split('.')[0]
                genome_db = os.path.join(blastables_outdir, base_name)
                outscaffold = f"{scaffolds_dir}/{id}_EVE.fasta"
                if run_blastdbcmd(id, genome_db, outscaffold): # if run_blastdbcmd returns True, "break" is executed -> exiting the innermost loop and moving back to process the next id
                    break  # but if blastdbcmd returns False -> next iteration of the innermost loop 

print('Scaffolds of interest:', scaffolds_of_interest)

Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, removing file.
Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 20000 nucleotides, remov

In [27]:
#len(scaffolds_of_interest)
scaffolds_of_interest

['NODE_381_length_27838_cov_14.2215',
 'NODE_352_length_29307_cov_12.7034',
 'NODE_45_length_194274_cov_4.45832',
 'NODE_16_length_371769_cov_13.539',
 'NODE_1_length_1589435_cov_13.0886',
 'NODE_10_length_626405_cov_4.34645',
 'NODE_70_length_144148_cov_190.155',
 'NODE_498_length_23434_cov_17.1238',
 'NODE_33_length_241831_cov_197.129',
 'NODE_89_length_113149_cov_3.22244',
 'NODE_33_length_241831_cov_197.129',
 'NODE_146_length_60639_cov_3.37158',
 'NODE_95_length_100189_cov_309.803',
 'NODE_122_length_74444_cov_387.187',
 'NODE_381_length_27838_cov_14.2215',
 'NODE_55_length_165510_cov_190.906',
 'NODE_58_length_162532_cov_4.15186',
 'NODE_98_length_98422_cov_62.5464',
 'NODE_43_length_199121_cov_7.50572',
 'NODE_2_length_969053_cov_7.62238',
 'NODE_37_length_230641_cov_4.40646',
 'NODE_11_length_515168_cov_3.2278',
 'NODE_63_length_153037_cov_4.46219',
 'NODE_244_length_37664_cov_11.8205',
 'NODE_104_length_85964_cov_12.9853',
 'NODE_66_length_150458_cov_3.12336',
 'NODE_3_length_

In [22]:
# melo by sedet - jen ids skafoldu nad inclusion treshold a 20000+
set_scaffolds_of_interest = set(scaffolds_of_interest) # set protoze v jednom scaffoldu je vic virovych orfu, takze se scaffold objevuje duplicitne
len(set_scaffolds_of_interest)

118

In [45]:
# ted potrebujeme srovnat scaffolds_of_interest s genome_scaffold_ids a pak to pouzijeme pro seqidlist
# ziskame dict: genomes a jejich filtered scaffolds (vice virovych orfu na jednom skafoldu -> redundance)
filtered_genome_scaffold_ids = {}
for id in set_scaffolds_of_interest:
    for genome, ids in genome_scaffold_ids.items():
        if id in ids:
            if genome not in filtered_genome_scaffold_ids:
                filtered_genome_scaffold_ids[genome] = []  # initialize the list if it doesn't exist  - takto se nebudou prepisovat values a pridaji se vsechny ids
            filtered_genome_scaffold_ids[genome].append(id)  #
print(filtered_genome_scaffold_ids)

{'Chic10_scaffolds': ['NODE_104_length_85964_cov_12.9853', 'NODE_66_length_150458_cov_3.12336', 'NODE_37_length_230641_cov_4.40646', 'NODE_43_length_199121_cov_7.50572', 'NODE_352_length_29307_cov_12.7034', 'NODE_33_length_241831_cov_197.129', 'NODE_26_length_312622_cov_7.43917', 'NODE_312_length_31651_cov_4.15834', 'NODE_498_length_23434_cov_17.1238', 'NODE_89_length_113149_cov_3.22244', 'NODE_9_length_666414_cov_4.15495', 'NODE_98_length_98422_cov_62.5464', 'NODE_146_length_60639_cov_3.37158', 'NODE_63_length_153037_cov_4.46219', 'NODE_16_length_371769_cov_13.539', 'NODE_94_length_101213_cov_13.992', 'NODE_70_length_144148_cov_190.155', 'NODE_11_length_515168_cov_3.2278', 'NODE_95_length_100189_cov_309.803', 'NODE_122_length_74444_cov_387.187', 'NODE_28_length_282822_cov_9.9159', 'NODE_10_length_626405_cov_4.34645', 'NODE_55_length_165510_cov_190.906', 'NODE_3_length_955702_cov_6.81221', 'NODE_45_length_194274_cov_4.45832', 'NODE_58_length_162532_cov_4.15186', 'NODE_13_length_427009_

In [46]:
sum(len(id) for id in filtered_genome_scaffold_ids.values())

118

# BLATSTN for repeats done! jeste vygenerpvat v outfmt formatu pro python - odentruji se vsechny vhodne repetice (2x identicke, vhodna delka) - musi se ulozit pod nazvem orfu

In [67]:
def blastn_repeats(query, output, database, seqid):
    command = ["blastn",
               "-query", str(query), 
               "-out", str(output),
               "-db", str(database), 
               "-seqidlist", str(seqid), 
               "-word_size", "8", #sensitivni command pro short direct repeats - zmenit pro TIRs
               "-dust", "no", 
               "-num_threads", "5", # zmenit na clusteru
               "-strand", "plus"] # sensitivni command pro short direct repeats - zmenit pro TIRs
    subprocess.run(command, check=True) 


output_dir = "/home/majnusova/all/projects/plv/data/alias_test/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
path_to_input = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
for genome, scaffold_ids in filtered_genome_scaffold_ids.items():
    database = f"{blastables_outdir}/{genome}"
    for scaffold_id in scaffold_ids:  
        query_file = f"{scaffold_id}_EVE.fasta"
        query = os.path.join(path_to_input, query_file)
        seqidlist_file = f"{scaffold_id}_seqidlist.txt"
        seqidlist_path = os.path.join(output_dir, seqidlist_file)
        with open(seqidlist_path, 'w') as file:
            file.write(scaffold_id + '\n')  # Writing the scaffold ID to the file

        output_file = f"{scaffold_id}_repeats.txt"
        output = os.path.join(output_dir, output_file)

    # Check if the query file exists to avoid the error
        if os.path.exists(query):
            blastn_repeats(query, output, database, seqidlist_path)
        else:
            print(f"Query file {query} does not exist, skipping this scaffold ID.")

KeyboardInterrupt: 

In [ ]:
----------------------------------------ok

## Turning the extracted scaffolds into single line files to make it possible to search for the repeats flanking the EVEs

In [ ]:
scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
singleline_dir = "/home/majnusova/all/projects/plv/data/eve_singleline_scaffolds/" 

for outscaffold in os.listdir(scaffolds_dir):  #os.listdir lists all files/directories; not paths
    outscaffold_path = os.path.join(scaffolds_dir, outscaffold) # paths to all the files listed
    with open(outscaffold_path, "r") as infile:
        lines = infile.readlines()
    header = lines[0]  # fasta header
    joined_lines = ''.join(line.strip() for line in lines[1:])  # Join all lines except the first

    output_file_path = os.path.join(singleline_dir, outscaffold)  
    with open(output_file_path, "w") as outfile:
        outfile.write(header) 
        outfile.write(joined_lines)  

## prepare blastn to find DRs (shortest one has 48 nt, longest one 101 nt) 

In [ ]:
kdyz se na jednom scaffoldu nachazi vice virovych proteinu (v hmmmout contig1_2, contig1_13...), tak dat id contigu do setu pro blastn,
aby se to zblastovalo jen jednou (cely skafold) a pak zkusit odentrovat vsechny vhodne repetice, co to najde a hledat pak jestli v nich je 
protein nalezeny hmmerem - kdyz jo, ulozit pod hmmer orf id jako soubor s EVE

## extract orfs between tirs - pujde v pohode i bez vymezeneho TSD. kontrola spravnosti elementu - v orfech musi byt protein identifikovany hmmerem hned na zacatku... 

In [ ]:
blastn -query visch13_scaf.txt -out visch13_repeats_dust_plus.txt -db Vischeria_C74_genome_v1 -word_size 8 -seqidlist seqid.txt -num_threads 5 -strand plus


outfmt pro python?